<a href="https://colab.research.google.com/github/yeonghun00/stock_public/blob/main/1.crawler/3.Naver_news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import datetime
import html
import re

class News:
  def __init__(self):
    self.headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'}

  def get_ranking(self, date='1', page=1):
    date = str(datetime.datetime.now().date().strftime("%Y%m%d")) if date == '1' else date
    page = str(page)
    url = 'https://finance.naver.com/news/news_list.nhn?mode=RANK&date=' + date + '&page=' + page
    result = requests.get(url, headers = self.headers)
    bs_obj = BeautifulSoup(result.content, "html.parser")

    div = bs_obj.find_all('div', {'class':'hotNewsList'})
    return [html.unescape(x['href']) for x in div[0].find_all('a')]

  def get_main(self, date='1'):
    date = str(datetime.datetime.now().date().strftime("%Y-%m-%d")) if date == '1' else date
    url = 'https://finance.naver.com/news/mainnews.nhn'
    result = requests.get(url, headers = self.headers)
    bs_obj = BeautifulSoup(result.content, "html.parser")
    return [x.find_all('a')[0]['href'] for x in bs_obj.find_all('dd', {'class':'articleSubject'})]

  # 시황 전망: 401
  # 기업 종목분석: 402
  # 해외 증시: 403
  # 채권 선물: 404
  # 공시 메모: 406
  # 환율: 429
  def get_focus(self, code=403, date='1', page=1):
    date = str(datetime.datetime.now().date().strftime("%Y%m%d")) if date == '1' else date
    url = 'https://finance.naver.com/news/news_list.nhn?mode=LSS3D&section_id=101&section_id2=258&section_id3='\
     + str(code) +  '&date=' + str(date) + '&page=' + str(page)
    result = requests.get(url, headers = self.headers)
    bs_obj = BeautifulSoup(result.content, "html.parser")
    temp = [html.unescape(x.find_all('a')[0]['href']) for x in bs_obj.find_all('dd', {'class':'articleSubject'})]
    # 업데이트 가능
    return [x.replace('§', '&sect') for x in temp]

  def get_article(self, url):
    url = 'https://finance.naver.com/' + url
    result = requests.get(url, headers = self.headers)
    bs_obj = BeautifulSoup(result.content, "html.parser")

    title = ' '.join(bs_obj.find_all('div', {'class': 'article_info'})[0].find_all('h3')[0].get_text().split())
    context = bs_obj.find_all('div', {'class': 'articleCont'})[0]
    try:
      subheading = context.find_all('strong')[0].get_text()
    except:
      subheading = ''
    t = ' '.join(context.get_text().split())
    text = t[:t.find('@')]
    return title, subheading, text

  # 특징주
  def get_search(self, keyword='%C6%AF%C2%A1%C1%D6', start_date='1', end_date='1'):
    start_date = str(datetime.datetime.now().date().strftime("%Y%m%d")) if start_date == '1' else start_date
    end_date = str(datetime.datetime.now().date().strftime("%Y%m%d")) if end_date == '1' else end_date

    li = []
    for page in range(1,1000):
      url = 'https://finance.naver.com/news/news_search.nhn?rcdate=&q=' + keyword +\
      '&sm=title.basic&pd=4&stDateStart=' + start_date + '&stDateEnd=' + end_date + '&page=' + str(page)
      result = requests.get(url, headers = self.headers)
      bs_obj = BeautifulSoup(result.content, "html.parser")
      hrefs = [x.find_all('a')[0]['href'] for x in bs_obj.find_all(re.compile('d*'), {'class':"articleSubject"})] 
      if not hrefs:
        break
      li += hrefs

    return li

In [2]:
from multiprocessing import Pool

news = News()
news_list = news.get_search('%C6%AF%C2%A1%C1%D6', '2021-06-11', '2021-06-11')
with Pool(50) as p:
    li = p.map(news.get_article, news_list)
li = np.array(li)

dic = {'Title':li[:,0], 'Subheading':li[:,1], 'Text':li[:,2]}
df = pd.DataFrame.from_dict(dic)
df

,Title,Subheading,Text
0,[특징주] 현대차·SK·포스코·효성 '수소 연합' 결성에 수소株 강세,,"현대차 등 4개 그룹, 수소기업협의체 설립 추진한다현대차그룹이 SK그룹, 포스코그룹..."
1,"[특징주] 엘비루셈, 코스닥 상장 첫날 하락…공모가보다는 8.6% 높아(종합)",,엘비루셈[서울IR 제공. 재판매 및 DB금지] (서울=연합뉴스) 박진형 기자 = 반...
2,"[특징주] 비트컴퓨터, 정부 비대면 진료 규제 개선에 관련 사업 부각",,비트컴퓨터의 주가가 상승세다.11일 오후 2시23분 현재 코스닥시장에서 비트컴퓨터는...
3,"[특징주] 로지시스, 이주열 총재 ""하반기 CBDC 모의실험 착수"" 발언에 상승세",,로지시스의 주가가 상승세다. 11일 오후 2시4분 기준 로지시스는 전 거래일 대비 ...
4,"[특징주]신풍제약, 코로나 종식 백신에 '먹는 치료제'도 필수…정부도 확보 노력",,[아시아경제 박형수 기자] 신풍제약이 이틀 연속으로 급등하고 있다. 미국 정부가 M...
5,"[특징주] 넥스턴바이오, 당뇨 1위 글로벌 기업과 협력… 8%↑",,사진=넥스턴바이오넥스턴바이오가 급등세다.11일 오후 1시54분 넥스턴바이오는 전 거...
6,"[특징주] 네오티스, 수소차 테마 강세에 24%↑",,11일 자동차 부품업체 네오티스가 수소차 테마 강세에 힘입어 52주 신고가를 갈아치...
7,"[특징주] 가온미디어, KT에 기가지니3 공급 소식에 '급등'",,사진=가온미디어토탈 솔루션 전문기업 가온미디어가 기가지니3 개발을 완료해 KT에 공...
8,"[특징주] 평화산업, 미래차 지원 기대감에 강세… 52주 신고가",,11일 자동차 부품업체 평화산업이 정부의 미래차 지원 발표에 수혜주로 부각되면서 강...
9,"[특징주] 넥스트사이언스, '나노젠' 코로나 백신 베트남 임상3상 최종 승인…대규모...",,[헤럴드경제=이호 기자] 넥스트사이언스가 2대 주주로 있는 베트남 제약사 나노젠이 ...
